<hr />
Example for Spark Streaming
<hr />

<hr />
The following command adds the pyspark to sys.path at runtime. If the pyspark is not on the system path by default. It also prints the path of the spark.
<hr />

In [1]:
import findspark

print(findspark.find())
findspark.init()

C:\Users\KanchanRushikeshKale\spark


<hr />
Create a Spark Session
<hr />

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("StructuredNetworkWordCount") \
    .getOrCreate()

<hr />
We use Netcat (a small utility found in most Unix-like systems) as a data server for providing the sentences to the streaming application. <br>

Run the below command in the terminal to start the data server on port 9999. <br>
nc -lk 9999 <br>

install netcat for windows<br>
https://nmap.org/download.html#windows <br>



<hr />
Import the packages required to split a sentence into words.
<hr />

In [3]:
from pyspark.sql.functions import explode
from pyspark.sql.functions import split 

<hr />
The variable "lines" is a DataFrame that represents an unbounded table containing the streaming text data. <br>
The wordCounts DataFrame groups the unique values in the Dataset and counts them. <br>
<hr />

In [4]:
# Create DataFrame representing the stream of input lines from connection to localhost:9999
lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

<hr />
The streaming computation is started in the background. <br>
The applications starts receiving data and computing the counts. <br>
Further, dispalys the complete set of counts to the console every time they are updated. <br>

After running the below cell, provide the input sentences in the data server terminal started earlier. <br>
<hr />

In [ ]:
 # Start running the query that prints the running counts to the console
query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()

query.awaitTermination()

In [ ]:
spark.stop()